# Order a Basemap Using the Orders API & SDK via AOI & Quad ID

## Overview ##
---
In this notebook, you will learn how to order a [Planet Basemap](https://developers.planet.com/docs/data/visual-basemaps/) using your [Area of Interest](https://developers.planet.com/apis/orders/basemaps/#order-basemaps-by-area-of-interest-aoi) (AOI) and a [Quad ID](https://developers.planet.com/apis/orders/basemaps/#order-basemaps-by-quad-ids-and-deliver-to-cloud). We will place this order via Planet's [Orders API](https://developers.planet.com/apis/orders/) using our Planet Python [SDK](https://planet-sdk-for-python-v2.readthedocs.io/en/latest/python/sdk-guide/). 

1. Get a Basemap ID using either [Planet Explorer](https://developers.planet.com/docs/apps/explorer/) or the [Basemap Viewer](https://www.planet.com/basemaps/#/mosaic/). 
2. Create a JSON order packet with order parameters.
3. Set up a session with the Planet SDK, and place the order, [delivering]((https://developers.planet.com/apis/orders/delivery/)) the resulting Basemap to a Google Cloud Storage bucket. 
4. Repeat steps 1-3 using quad IDs. 

## Order a Basemap using an AOI
---
### First, find a Basemap to order
You can get a Basemap ID from Planet Explorer or the Basemap Viewer.

#### Option 1: Get a basemap ID from Planet Explorer
You'll need a basemap ID to identify what basemap to download.

To pick a basemap, you can open [Planet Explorer](https://www.planet.com/explorer/#) (you can learn more about Planet Explorer [here](https://developers.planet.com/docs/apps/explorer/)) and select a basemap from the dropdown menu in the lower left corner. (If you don't have access to basemaps, you'll see a message confirming in the lower right corner. Select "Get Access" to sign up for Basemaps or speak to your customer service manager.)

Once you've selected a Basemap, the Explorer updates to include that basemap in the view.

Select the search icon and time cadence, for example `monthly` or `quarterly`. Then select one of the resulting basemap IDs, such as `global_monthly_2022_01_mosaic`. That's the value you'll be passing into the Orders API. 

#### Option 2: Get a basemap ID from the Basemap Viewer

To pick a basemap, you can open the [Basemap Viewer](https://www.planet.com/basemaps/#/mosaic/) (you can learn more about the Basemap Viewer [here](https://developers.planet.com/docs/apps/basemapsviewer/)) and select a basemap from the left sidebar menu, using the filter to narrow down the basemap you want. 

Once you've selected a basemap, and selected the right arrow >, the basemap ID displays, such as `global_monthly_2022_01_mosaic`. That's the value you'll be passing into the Orders API.

### Import Planet and Related Packages

---

Make sure you have Planet's Python SDK properly downloaded. You can find out more about this [here](https://developers.planet.com/docs/pythonclient/). Find your [API key](https://developers.planet.com/quickstart/apis/).

Next set up a session by importing needed Python packages, pulling in your API Key, and make an initial request (to retrieve the Orders API parameters) to confirm a connection with the server.

Create a Google Cloud Platform (GCP) storage bucket with these [instructions](https://cloud.google.com/storage/docs/creating-buckets). This is where we are delivering the data. Creating the bucket will create the credentials for you, and you can convert it to Base64 using these [instructions](https://cloud.google.com/vision/docs/base64) and save this as an environment variable. 

In [15]:
import planet
import os

In [16]:
PL_API_KEY = os.environ.get('PL_API_KEY')
GCP_CREDENTIALS = os.environ.get('GCP_CREDENTIALS')
auth = planet.Auth.from_key(PL_API_KEY)

### Create an order packet
---
Package up the details of your order in a [JSON object](https://developers.planet.com/apis/orders/basemaps/#example-order-query-json-block) and make a POST request, passing in the Orders URL, your JSON, your API key, and the content-type header. We are [delivering](https://developers.planet.com/apis/orders/delivery/) this order to a Google Cloud Storage bucket. In this request, we are also using the tools [merge](https://developers.planet.com/apis/orders/tools/#merge) and [clip](https://developers.planet.com/apis/orders/tools/#clip). 

In [17]:
order_params = {
    "name": "basemap order with geometry",
    "source_type": "basemaps",
    "order_type": "partial",
    "products": [
        {
            "mosaic_name": "global_monthly_2022_01_mosaic",
            "geometry": {
                "type": "Polygon",
                "coordinates": [
                   [
                       [4.607406, 52.353994],
                       [4.680005, 52.353994],
                       [4.680005, 52.395523],
                       [4.607406, 52.395523],
                       [4.607406, 52.353994]
                   ]
                ]
            }
        }
    ],
    "tools": [
        {"merge": {}},
        {"clip": {}}
    ],
    "delivery": {
        "google_cloud_storage": {
            "bucket": "devrel-notebooks",
            "credentials": GCP_CREDENTIALS,
            "path_prefix": "basemaps-to-cloud/"
        }
    }
}

### Create a session with SDK and poll for success

Here, we are creating an order using the SDK. Then we are waiting for the order to be delievered to our GCP storage bucket. Note that this can take some time.

In [18]:
async def create_and_deliver_order(order_params, client):
    '''Create an order and wait for it to delivered

    Parameters:
        order_params: An order request
        client: An Order client object
    '''
    with planet.reporting.StateBar(state='creating') as reporter:
        # Place an order to the Orders API
        order = await client.create_order(order_params)
        reporter.update(state='created', order_id=order['id'])
        # Wait while the order is being completed
        await client.wait(order['id'], callback=reporter.update_state,
                          max_attempts=0)

In [19]:
async with planet.Session() as ps:
    # The Orders API client
    client = ps.client('orders')
    # Create the order and deliver it to GCP
    await create_and_deliver_order(order_params, client)

00:11 - order e912671f-cda7-473e-a70d-f9ce72dd3194 - state: running


CancelledError: 

### Check your results

After a few simple steps, we have a basemap delivered to our Google Cloud Platform bucket using the Planet Python SDK.

## Use Basemaps API to get the Basemap and quad IDs
---

### What is a Quad ID? 

A quad ID is a unique identifer to access a square tile of imagery of the Earth's surface, whereas a Basemap is a combination of squares. 

Use the Basemap APIs to retrieve the basemap you want and the quad IDs you are looking for. You can get the base URL you'll need to communicate with the Basemap API service:

1. Go to the [Basemap API reference](https://developers.planet.com/docs/basemaps/reference/) at https://developers.planet.com/docs/basemaps/reference/. 
2. Under List Mosaics, select GET /mosaics.

A dropdown UI element appears with the URL to use:

https://api.planet.com/basemaps/v1/mosaics


In [20]:
import requests
from requests.auth import HTTPBasicAuth
import json
import math
from urllib3.util.retry import Retry
from requests.adapters import HTTPAdapter

In [21]:
auth = HTTPBasicAuth(PL_API_KEY, '')
session = requests.Session()

# Authenticate
session.auth = (PL_API_KEY, '')

#### Search for the mosaic ID to get its quad IDs

Here, we are using the full name of the mosaic, but with the `name__contains` field, you can use a partial name based on the names of the mosaics available through your account.

Declare the search string for `requests.get()`. You will use the dictionary you create with the mosaic name to pass to the requests.get() function.


This code sets the URL and headers for an HTTP request to Planet's Basemap API and sends a GET request with the specified search parameters. If the response returns the correct status, the JSON response is loaded into a Python dictionary with mosaics information from which we'll extract quad ID information.

In [22]:
def consume_pages(session, url, params, key='items'):
    """Handle paginated URLs"""
    while True:
        response = session.get(url, params=params)
        response.raise_for_status()
        body = response.json()
        for item in body[key]:
            yield item

        if '_next' in body['_links']:
            url = body['_links']['_next']
        else:
            break

In [23]:
BASEMAP_API_URL = 'https://api.planet.com/basemaps/v1/mosaics'

# Wait and retry automatically on 429 "slow down" responses
retries = Retry(total=10, backoff_factor=1, status_forcelist=[429])
session.mount('https://', HTTPAdapter(max_retries=retries))


basemap_params = {
    'name__is': 'ps_biweekly_sen2_normalized_analytic_subscription_2023-03-20_2023-04-03_mosaic'
}

session.headers.update({'content-type': 'application/json'})

# basemap_service_response = session.get(
#     url=BASEMAP_API_URL, params=basemap_params)

# if basemap_service_response.status_code < 400:
#     basemaps = basemap_service_response.json()
# else:
#     basemap_service_response.raise_for_status()

This code retrieves the ID and bounding box coordinates of the mosaic we loaded above and uses these to construct an API request to retrieve the corresponding quad IDs for a specific area.

In [24]:
# We expect only a single result with a name__is query, but it follows the 
# same paginated pattern (and name__contains often has multiple pages)
mosaic, = consume_pages(session, BASEMAP_API_URL, basemap_params, 'mosaics')

quad_params = {
    'bbox': ','.join(map(str, mosaic['bbox']))
}
quads_url = "{}/{}/quads".format(BASEMAP_API_URL, mosaic['id'])
quads = consume_pages(session, quads_url, quad_params)

# Make sure we only get quads that we have download access to
quad_ids = [item['id'] for item in quads if 'download' in item['_links']]

### Create an order packet
---
Create an order packet with details including the mosaic name and quad IDs, and the tools and delivery method. In this example, we are using the tools [bandmath](https://developers.planet.com/apis/orders/tools/#band-math) and [reproject](https://developers.planet.com/apis/orders/tools/#reproject). 

#### Bandmath 

The bandmath tool allows you to apply band math expressions to the bands of your input files to produce derived outputs and indices for analysis. Note that `b5` in this example is being assigned to [NDVI](https://en.wikipedia.org/wiki/Normalized_difference_vegetation_index). 

#### Reproject

Reprojecting a basemap is necessary when the original map projection does not accurately represent the Earth's surface in a specific region or area. This can happen when the imagery for the basemap comes from a place far from the equator. In the example shown in this notebook, there is not a significant change in circumference of the earth from the equator, but it is always best to check using the code below. 

Find the latitude of the area of interest and input it as a parmameter. The code below finds the correct resolution for the area of interest.

In [28]:
def calculate_resolution(latitude):
    '''Calculate the resolution of a basemap at a given latitude

    Parameters: latitude (float): latitude at location of interest [degrees]
    Return (float): resolution of basemap at given latitude [degrees per pixel]
    '''
    earth_radius = 6378137.0  # meters
    native_resolution = 3  # meters per pixel
    circumference_of_earth = 2.0 * math.pi * (earth_radius)
    meters_per_degree_at_latitude = math.cos(math.radians(
        latitude)) * circumference_of_earth / 360.0
    resolution_at_latitude = native_resolution / meters_per_degree_at_latitude
    return resolution_at_latitude

In [29]:
resolution_at_latitude = calculate_resolution(37.0) # input latitude as a perameter here

In [30]:
order_params = {
    "name": "basemap order with quad_ids",
    "source_type": "basemaps",
    "products": [
        {
            "mosaic_name": "ps_biweekly_sen2_normalized_analytic_subscription_2023-03-20_2023-04-03_mosaic",
            "quad_ids": quad_ids
        }
    ],
    "tools": [
        {
            "reproject": {
                "projection": "EPSG:4326",
                "resolution": resolution_at_latitude,
                "kernel": "cubic"
            }
        },
        {
            "bandmath": {
                "b1": "b1",
                "b2": "b2",
                "b3": "b3",
                "b4": "b4",
                "b5": "(b4-b3)/(b4+b3)",
                "pixel_type": "32R"
            }
        }
    ],
    "delivery": {
        "google_cloud_storage": {
            "bucket": "devrel-notebooks",
            "credentials": GCP_CREDENTIALS,
            "path_prefix": "basemaps-to-cloud/",
        }
    }
}

### Create and deliver order with the SDK



In [31]:
async with planet.Session() as ps:
    # The Orders API client
    client = ps.client('orders')
    # Create the order and deliver it to GCP
    await create_and_deliver_order(order_params, client)

00:01 - order  - state: creating


BadQuery: {"field":null,"general":[{"message":"Unable to accept order: validation of input quads failed: Get \"https://mosaics.prod.planet-labs.com/basemaps/v1/mosaics/29899c94-d4fe-47fa-9661-dd36b588b22f/quads/1207-1375,1208-1375,1209-1375,1211-1375,1212-1375,1213-1375,1214-1375,1215-1375,1216-1375,1207-1374,1208-1374,1209-1374,1210-1374,1211-1374,1212-1374,1213-1374,1214-1374,1215-1374,1216-1374,1217-1374,1199-1373,1200-1373,1201-1373,1202-1373,1203-1373,1204-1373,1205-1373,1207-1373,1208-1373,1209-1373,1210-1373,1211-1373,1212-1373,1213-1373,1214-1373,1215-1373,1216-1373,1217-1373,1218-1373,1199-1372,1200-1372,1201-1372,1202-1372,1203-1372,1204-1372,1205-1372,1206-1372,1207-1372,1208-1372,1209-1372,1210-1372,1211-1372,1212-1372,1213-1372,1214-1372,1215-1372,1216-1372,1217-1372,1218-1372,1162-1371,1163-1371,1164-1371,1165-1371,1166-1371,1167-1371,1168-1371,1169-1371,1170-1371,1171-1371,1172-1371,1173-1371,1198-1371,1199-1371,1200-1371,1201-1371,1202-1371,1203-1371,1204-1371,1205-1371

### Check your results

After a few simple steps, we have a basemap delivered to our Google Cloud Platform bucket using the Planet Python SDK.  For more information, check out the SDK docs [here](https://planet-sdk-for-python-v2.readthedocs.io/en/latest/python/sdk-guide/). 